In [16]:
%pip install keras
%load_ext autoreload
%autoreload 2

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [18]:
from sklearn import svm
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import tensorflow as tf

(train_ds, test_ds) = tfds.load(
    'deep_weeds',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
)

In [19]:
train_ds_numpy = [(instance.numpy(), label.numpy()) for instance, label in train_ds if label < 8]
test_ds_numpy = [(instance.numpy(), label.numpy()) for instance, label in test_ds if label < 8]

random_train_indices = np.random.choice(len(train_ds_numpy), size=200, replace=False)
random_test_indices = np.random.choice(len(test_ds_numpy), size=50, replace=False)

X_train_original = [[instance for instance, _ in train_ds_numpy][i] for i in random_train_indices]
y_train = [[label for _, label in train_ds_numpy][i] for i in random_train_indices]

X_test_original = [[instance for instance, _ in test_ds_numpy][i] for i in random_test_indices]
y_test = [[label for _, label in test_ds_numpy][i] for i in random_test_indices]

print(len(X_train_original))
print(len(X_test_original))

200
50


In [20]:
# from skimage import feature
# import cv2

# print(np.array(X_test_original).shape)

# sigma = 1.5
# X_train_image = list(map(lambda x: feature.canny(cv2.cvtColor(x, cv2.COLOR_RGB2GRAY), sigma=sigma), X_train_original))
# X_test_image = list(map(lambda x: feature.canny(cv2.cvtColor(x, cv2.COLOR_RGB2GRAY), sigma=sigma), X_test_original))

In [21]:
# fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 3))

# ax[0].imshow(X_train_image[0], cmap='gray')
# ax[0].set_title(r'from X_train with $\sigma=1.5$', fontsize=20)

# ax[1].imshow(X_test_image[0], cmap='gray')
# ax[1].set_title(r'from X_test with $\sigma=1.5$', fontsize=20)

In [22]:
# Instantiate convolutional base
# Reference: https://towardsdatascience.com/transfer-learning-from-pre-trained-models-f2393f124751
from tensorflow.keras.applications import VGG16

img_width, img_height = X_train_original[0].shape[0:2]
conv_base = VGG16(weights='imagenet', 
                  include_top=False,
                  input_shape=(img_width, img_height, 3))  # 3 = number of channels in RGB pictures

conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [23]:
# Extract features
import os, shutil
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 32

def extract_features(x, y):
    sample_count = len(y)
    
    features = np.zeros(shape=(sample_count, 8, 8, 512))  # Must be equal to the output of the convolutional base
    labels = np.zeros(shape=(sample_count))
    # Preprocess data
    generator = datagen.flow(np.array(x), np.array(y), batch_size = batch_size)
    # Pass data through convolutional base
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        print(features_batch.shape)
        features[i * batch_size: (i + 1) * batch_size] = features_batch
        labels[i * batch_size: (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels
    
train_features, train_labels = extract_features(X_train_original, y_train)  # Agree with our small dataset size
# validation_features, validation_labels = extract_features(validation_dir, validation_size)
test_features, test_labels = extract_features(X_test_original, y_test)

(32, 8, 8, 512)
(32, 8, 8, 512)
(32, 8, 8, 512)
(32, 8, 8, 512)
(32, 8, 8, 512)
(32, 8, 8, 512)
(8, 8, 8, 512)
(32, 8, 8, 512)
(18, 8, 8, 512)


In [24]:
X_train = list(map(lambda x: x.flatten(), train_features))
y_train = train_labels
X_test = list(map(lambda x: x.flatten(), test_features))
y_test = test_labels

print(train_labels.shape)
print("feature size now:", X_train[0].shape) # orginal feature is of dimension 196608

(200,)
feature size now: (32768,)


In [43]:
linear = svm.SVC(kernel='linear', C=1, decision_function_shape='ovo').fit(X_train, y_train)
linear_pred = linear.predict(X_test)
accuracy_lin = linear.score(X_test, y_test)
print(accuracy_lin)
linear = svm.SVC(kernel='linear', C=0.00025, decision_function_shape='ovo').fit(X_train, y_train)
linear_pred = linear.predict(X_test)
accuracy_lin = linear.score(X_test, y_test)
print(accuracy_lin)

0.52
0.34


In [53]:
from run_algo_with_kfold import kfold_cross_validation

Cs = [0.0024, 0.0026]
gammas = ['scale']
k = 5
final_accuracies = []
for C in Cs:
  for gamma in gammas:
    final_accuracy = kfold_cross_validation(k, X_train, y_train, 'svm', {'kernel': 'linear', 'C': C, 'gamma': gamma})
    final_accuracies.append(final_accuracy)

Running 5-fold cross validation for svm with {'kernel': 'linear', 'C': 0.0024, 'gamma': 'scale'}
Obtained split accuracy of: 0.625
Obtained split accuracy of: 0.425
Obtained split accuracy of: 0.5
Obtained split accuracy of: 0.375
Obtained split accuracy of: 0.475
Completed 5-fold cross validation for svm with {'kernel': 'linear', 'C': 0.0024, 'gamma': 'scale'}
Obtained average accuracy of: 0.48

Running 5-fold cross validation for svm with {'kernel': 'linear', 'C': 0.0026, 'gamma': 'scale'}
Obtained split accuracy of: 0.625
Obtained split accuracy of: 0.425
Obtained split accuracy of: 0.5
Obtained split accuracy of: 0.375
Obtained split accuracy of: 0.475
Completed 5-fold cross validation for svm with {'kernel': 'linear', 'C': 0.0026, 'gamma': 'scale'}
Obtained average accuracy of: 0.48



In [30]:
linear = svm.SVC(kernel='linear', C=1, decision_function_shape='ovo').fit(X_train, y_train)
linear_pred = linear.predict(X_test)
accuracy_lin = linear.score(X_test, y_test)
print("Accuracy Linear Kernel:", accuracy_lin)

Accuracy Linear Kernel: 0.672


In [31]:
rbf = svm.SVC(kernel='rbf', gamma=0.1, C=1, decision_function_shape='ovo').fit(X_train, y_train)
rbf_pred = rbf.predict(X_test)
accuracy_rbf = rbf.score(X_test, y_test)
print("Accuracy Radial Basis Kernel:", accuracy_rbf)

Accuracy Radial Basis Kernel: 0.104


In [32]:
poly = svm.SVC(kernel='poly', degree=3, C=1, decision_function_shape='ovo').fit(X_train, y_train)
poly_pred = poly.predict(X_test)
accuracy_poly = poly.score(X_test, y_test)
print("Accuracy Polynomial Kernel:", accuracy_poly)

Accuracy Polynomial Kernel: 0.472


In [33]:
sig = svm.SVC(kernel='sigmoid', C=1, decision_function_shape='ovo').fit(X_train, y_train)
sig_pred = sig.predict(X_test)
accuracy_sig = sig.score(X_test, y_test)
print("Accuracy Sigmoid Kernel:", accuracy_sig)

Accuracy Sigmoid Kernel: 0.176
